In [1]:
%load_ext autoreload
%autoreload now

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-08137aa2-e69b-5e74-8390-7997329b1336"

# Download and convert data

In [3]:
from document_segmentation.pagexml.annotations.generale_missiven import GeneraleMissiven
from document_segmentation.settings import (
    GENERALE_MISSIVEN_DOCUMENT_DIR,
    GENERALE_MISSIVEN_SHEET,
)

N = 5

sheet = GeneraleMissiven(GENERALE_MISSIVEN_SHEET)
documents = sheet.download(GENERALE_MISSIVEN_DOCUMENT_DIR, N)

# Load Data

In [4]:
%autoreload now

In [5]:
TRAINING_DATA = 0.8

In [6]:
from document_segmentation.model.dataset import DocumentDataset

dataset = DocumentDataset.from_documents(documents)
dataset.shuffle()

len(dataset)

Skipping row with inventory number 1171 due to reason: 'Niet gedigitaliseerd.'
Skipping row with inventory number 2770 due to reason: 'Niet gedigitaliseerd.'
Skipping row with inventory number 2770 due to reason: 'Niet gedigitaliseerd.'
Skipping row with inventory number 2770 due to reason: 'Niet gedigitaliseerd.'
Skipping row with inventory number 2911 due to reason: 'Niet gedigitaliseerd.'


909

In [7]:
dataset._class_counts()

Counter({<Label.IN: 1>: 189343, <Label.BEGIN: 0>: 905, <Label.END: 2>: 898})

In [8]:
dataset.class_weights()

[1.0033112582781456, 0.004800785871218523, 1.0111234705228032, 909.0]

In [9]:
training_data, test_data = dataset.split(TRAINING_DATA)

In [10]:
training_data._class_counts()

Counter({<Label.IN: 1>: 153509, <Label.BEGIN: 0>: 724, <Label.END: 2>: 717})

In [11]:
test_data._class_counts()

Counter({<Label.IN: 1>: 35834, <Label.BEGIN: 0>: 181, <Label.END: 2>: 181})

# Train Model

In [12]:
BATCH_SIZE = 64
EPOCHS = 3
WEIGHTS = dataset.class_weights()

In [13]:
from document_segmentation.model.page_sequence_tagger import PageSequenceTagger

tagger = PageSequenceTagger()

In [14]:
tagger._device

'mps'

In [15]:
tagger

PageSequenceTagger(
  (_page_embedding): PageEmbedding(
    (_region_model): RegionEmbeddingSentenceTransformer(
      (_transformer_model): SentenceTransformer(
        (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
        (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
      )
      (_region_type): Embedding(9, 16)
      (_linear): Linear(in_features=784, out_features=512, bias=True)
    )
    (_rnn): LSTM(512, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
    (_linear): Linear(in_features=512, out_features=256, bias=True)
  )
  (_rnn): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (_linear): Linear(in_features=512, out_feature

In [16]:
tagger.train_(training_data, EPOCHS, BATCH_SIZE, WEIGHTS)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: carschno. Use `wandb login --relogin` to force relogin


Training:   7%|▋         | 196/2873 [32:41<7:26:35, 10.01s/batch]   


KeyboardInterrupt: 

# Evaluate Model

In [ ]:
import sys

precision, recall, f1, accuracy = tagger.eval_(test_data, BATCH_SIZE, sys.stdout)

Predicted	Actual	Page ID	Text	Scores


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  1.54batch/s]

OUT	BEGIN	NL-HaNA_1.04.02_1068_0041.jpg	trian P den W. Beer.; Erntfeste wijse voorsinnige 	[8.121613063849509e-05, 7.864946383051574e-05, 0.00025079032639041543, 0.9995893836021423]
OUT	IN	NL-HaNA_1.04.02_1068_0042.jpg	Alsoo door dese questie In langen tijt geen peper 	[2.0223884348524734e-05, 1.3506860341294669e-05, 3.1163774110609666e-05, 0.9999351501464844]
OUT	IN	NL-HaNA_1.04.02_1068_0043.jpg	Hoe den Pangoran d'engelssen, om ons te pickeren, 	[1.1302064194751438e-05, 6.2080812313070055e-06, 1.1346785868227016e-05, 0.9999711513519287]
OUT	IN	NL-HaNA_1.04.02_1068_0044.jpg	geschieden, Soo wij voor de Compste vand' engelsse	[9.256930752599146e-06, 4.695950792665826e-06, 7.506770998588763e-06, 0.9999785423278809]
OUT	IN	NL-HaNA_1.04.02_1068_0045.jpg	Doen het nu gebeurde dal goet vonden, onde macht t	[8.714954674360342e-06, 4.281552264728816e-06, 6.358447990351124e-06, 0.9999806880950928]
OUT	IN	NL-HaNA_1.04.02_1068_0046.jpg	achternemen, ende te beneficeren. ter besten van g	[8.567697477

In [ ]:
import csv

from document_segmentation.pagexml.datamodel.label import Label

writer = csv.DictWriter(
    sys.stdout, fieldnames=["Metric"] + [label.name for label in Label], delimiter="\t"
)

writer.writeheader()

for metric in precision, recall, f1:
    assert metric.average is None

    scores: list[float] = metric.compute().tolist()
    writer.writerow(
        {"Metric": metric.__class__.__name__}
        | {label.name: f"{score:.4f}" for label, score in zip(Label, scores)}
    )

assert accuracy.average is not None

print(
    f"{accuracy.__class__.__name__} ({accuracy.average} average):\t{accuracy.compute().item():.4f}",
    file=sys.stdout,
)

        [1],
        [2]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.


Metric	BEGIN	IN	END	OUT
MulticlassPrecision	0.0000	0.0000	0.0000	0.0000
MulticlassRecall	0.0000	0.0000	0.0000	0.0000
MulticlassF1Score	0.0000	0.0000	0.0000	0.0000
MulticlassAccuracy (micro average):	0.0000
